In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [3]:
# 定义了一个继承自 PyTorch 的nn.Module的神经网络类
class NeuralNetwork(nn.Module):
    def __init__(self):
        # 调用父类的构造函数
        super().__init__()
        # 创建一个展平层，用于将二维图像转换为一维向量
         # 首先通过nn.Flatten()将 28×28 的输入图像展平为 784 个元素的一维向量
        self.flatten = nn.Flatten()
        # 定义了一个包含多个线性层和激活函数的序列结构
        self.linear_relu_stack = nn.Sequential(
            # 第一个线性层 (nn.Linear(28*28, 512)) 将 784 维输入转换为 512 维输出
            nn.Linear(28 * 28, 512),
            # 接着是 ReLU 激活函数 (nn.ReLU())，引入非线性特性
            nn.ReLU(),
            # 第二个线性层 (nn.Linear(512, 512)) 保持 512 维的特征空间
            nn.Linear(512, 512),
            # 再次使用 ReLU 激活函数
            nn.ReLU(),
            # 最后一个线性层 (nn.Linear(512, 10)) 将 512 维特征映射到 10 个输出，对应 10 个数字 (0-9) 的类别
            nn.Linear(512, 10),
        )

    def forward(self, x):
        '''
        forward方法定义了数据在网络中的流动方式
            首先将输入x通过展平层
            然后通过定义好的linear_relu_stack序列
            返回最终的 logits（未经过 softmax 激活的原始输出）
        :param x: 输入
        :return: 最终的 logits（未经过 softmax 激活的原始输出）
        '''

        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
# 定义 NeuralNetwork 实例，
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
# 使用torch.rand()生成一个随机张量作为输入数据, 张量形状为(1, 28, 28)，表示：1 个样本，每个样本是 28×28 的图像
# device=device将数据移动到指定的计算设备（可能是 CPU 或 GPU）
X = torch.rand(1, 28, 28, device=device)
# 将输入数据X传入神经网络模型model进行前向计算,得到的logits是模型的原始输出
# 输出（未经过归一化的分数），形状为(1, 10)（1 个样本，10 个类别）
logits = model(X)
# 使用 Softmax 函数将 logits 转换为概率分布
# dim=1表示在类别维度上进行归一化（确保每个样本的 10 个类别概率之和为 1）
# 得到的pred_probab是每个类别的预测概率，形状仍为(1, 10)
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
# 使用argmax(1)找到概率最大的类别索引（在类别维度上）
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[0.0962, 0.1028, 0.0940, 0.1049, 0.0966, 0.0987, 0.1071, 0.0898, 0.1080,
         0.1018]], device='mps:0', grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([8], device='mps:0')


In [9]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image)
print(flat_image.size())

tensor([[0.4706, 0.7414, 0.3911,  ..., 0.9447, 0.3018, 0.8735],
        [0.8664, 0.0698, 0.8932,  ..., 0.6892, 0.3485, 0.0354],
        [0.6275, 0.4386, 0.0691,  ..., 0.4209, 0.7809, 0.8359]])
torch.Size([3, 784])


In [15]:
layer1 = nn.Linear(in_features= 28 * 28, out_features= 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [16]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1223,  0.0242,  0.2426,  0.0364, -0.1334,  0.3675,  0.6220,  0.1916,
         -0.3052,  0.0701, -0.1356, -0.0931,  0.1212,  0.2348,  0.1169,  0.3111,
          0.3042,  0.2884, -0.6837,  0.2741],
        [ 0.0785, -0.1754,  0.4646,  0.2398, -0.6994,  0.1631,  0.4435,  0.2106,
         -0.3928, -0.1242, -0.2128, -0.1147,  0.0433,  0.1571, -0.2111,  0.2799,
          0.1298,  0.0816, -0.6057,  0.4190],
        [ 0.1813,  0.4338,  0.2983,  0.1512, -0.3722,  0.0937,  0.9095, -0.1226,
          0.0574,  0.0598, -0.0434, -0.2293, -0.0790,  0.2621, -0.0398,  0.2389,
          0.2833,  0.1925, -0.4494,  0.2398]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1223, 0.0242, 0.2426, 0.0364, 0.0000, 0.3675, 0.6220, 0.1916, 0.0000,
         0.0701, 0.0000, 0.0000, 0.1212, 0.2348, 0.1169, 0.3111, 0.3042, 0.2884,
         0.0000, 0.2741],
        [0.0785, 0.0000, 0.4646, 0.2398, 0.0000, 0.1631, 0.4435, 0.2106, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0433, 0.1571, 0.00

In [17]:
seq_models = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10),
)
input_image = torch.rand(3, 28, 28)
logits = seq_models(input_image)

In [18]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [20]:
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0005,  0.0004, -0.0255,  ..., -0.0356,  0.0109,  0.0356],
        [-0.0184,  0.0276,  0.0258,  ...,  0.0347, -0.0200, -0.0089]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0052,  0.0122], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0065,  0.0316,  0.0246,  ...,  0.0128,  0.0423, -0.0263],
        [ 0.0161,  0.0121,  0.0193,  ...,  0.0188, -0.0219, -0.0322]],
       device='mps:0', grad_fn=<Slice